In [1]:
!pip install imutils

In [2]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [3]:
# Set paths to root, train, test and validation directories
_rootDir = '../input/braintumor/Brain/'
#os.listdir(_rootDir)
_trainDir = _rootDir + 'train/'
_valDir = _rootDir + 'validation/'
_testDir = _rootDir + 'test/'

In [4]:
# Function to preprocess and store images and labels
def load(paths, verbose=-1):
    '''
        return:
            _imageList: A list of Images
            labels: A list of corresponding Labels
        args:
            paths: Path to main directory 
            * Must contain images belonging to a class as a single folder
    '''
    _imageList = list()
    _labelList = list()
    
    for (i, _imgPath) in enumerate(paths):
        
        img = cv2.imread(_imgPath, cv2.IMREAD_GRAYSCALE)
        # Set the desired height and weight of image data
        _dimensions = (32, 32)
        
        # resize image
        _image = cv2.resize(img, _dimensions, interpolation = cv2.INTER_AREA)
        numpyImage = np.array(_image).flatten()
        
        # rescale the image
        mupyImage = numpyImage/255
        label = _imgPath.split(os.path.sep)[-2]
        #print(label)
        
        _imageList.append(mupyImage)
        _labelList.append(label)
        
    # return a tuple of the _imageList and _labelList
    return _imageList, _labelList

In [5]:
def one_hot_encoding(path):
    _path = list(paths.list_images(path))
    _data, _labelList = load(_path, verbose=10000)
    _lblzr = LabelBinarizer()
    _labels = _lblzr.fit_transform(_labelList)
    
    return _data, _labels

In [6]:
# Preprocess training data
_trainX, _trainY = one_hot_encoding(_trainDir)
#print(len(_trainX))
#print(type(_trainY))

In [7]:
# Preprocess validation and test data
_valX, _valY = one_hot_encoding(_valDir)

_testX, _testY = one_hot_encoding(_testDir)

In [8]:
def create_clients(image_list, label_list, num_clients):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    _clientNames = []
    for i in range(num_clients):
        _client = '{}_{}'.format('client', i+1)
        _clientNames.append(_client)

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    
    _clientShards =[]
    for i in range(0, size*num_clients, size):
        _shard = data[i: i+size]
        _clientShards.append(_shard)
    
    #number of clients must equal number of shards
    if len(_clientShards) == len(_clientNames):
        _clientDict = {}
        for i in range(len(_clientNames)):
            _clientDict[_clientNames[i]] = _clientShards[i]
        return _clientDict
    else:
        return 0
    

In [9]:
clients = create_clients(_trainX, _trainY, num_clients = 7)

In [10]:
def batch_data(data_shard, bs=64):
    
    # Create tensorflow datasets from each client's data shards
    data, label = zip(*data_shard)
    #print("----------------"+str(type(data)))
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [11]:
# Create batches for training data
clients_batched = dict()

for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

In [12]:
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((_testX, _testY)).batch(len(_testY))

#process and batch the test set  
val_batched = tf.data.Dataset.from_tensor_slices((_valX, _valY)).batch(len(_valY))

In [13]:
# Define the hyperparameters
lr = 0.01 
_globalEpochs = 200
loss='categorical_crossentropy'
metrics = ['accuracy'] 
optimizer = SGD(lr=lr, 
                decay=lr / _globalEpochs, 
                momentum=0.9
               )   
activationFunction = "relu"

In [14]:
# Class for model architecture
class MLP:
    @staticmethod
    def _buildModel(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

In [15]:
def test_model(X_test, Y_test,  model, comm_round):
    
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)

    loss = cce(Y_test, logits)

    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('test_acc: {:.3%} | test_loss: {}'.format(acc, loss))
    return acc, loss

In [16]:
central_data = tf.data.Dataset.from_tensor_slices((_trainX,_trainY)).shuffle(len(_trainY)).batch(640)
central_object = MLP()

central_model = central_object._buildModel(1024,4)

central_model.compile(loss=loss,
                     optimizer=optimizer,
                     metrics=metrics)

central_model.fit(central_data, epochs=100,verbose=0)

for(X_test,Y_test) in test_batched:
    _acc, _loss = test_model(X_test,Y_test, central_model, 1)


In [17]:
central_data = tf.data.Dataset.from_tensor_slices((_trainX,_trainY)).shuffle(len(_trainY)).batch(640)
central_object = MLP()

central_model = central_object._buildModel(1024,4)

central_model.compile(loss=loss,
                     optimizer=optimizer,
                     metrics=metrics)

central_model.fit(central_data, epochs=200,verbose=0)

for(X_test,Y_test) in test_batched:
    _acc, _loss = test_model(X_test,Y_test, central_model, 1)